<a href="https://colab.research.google.com/github/RihaChri/ImageClassificationBreastCancer/blob/main/CNNBreatCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from re import search
from tensorflow import keras
#-----------------------load images---------------------------------------------
#imageDir=os.path.abspath('/content/drive/MyDrive/Colab Notebooks/ImageClassificationBreastCancerCNN/images')
imageDir=os.path.abspath('/content/drive/MyDrive/Colab Notebooks/ImageClassificationBreastCancerCNN/images/validation')
subDirList=[x[0] for x in os.walk(imageDir)]

fileList=[]
for folder in subDirList: 
  for file in glob.glob(folder+'/*.png'):
    fileList.append(file)

images=[]
y=np.zeros(len(fileList))
for i in range(len(fileList)):
  image = cv2.imread(fileList[i])
  image = cv2.resize(image, (700, 460), cv2.INTER_LINEAR)
  images.append(image)
  if search('SOB_M', fileList[i]): y[i]=1
  elif search('SOB_B', fileList[i]): y[i]=0
  else: raise Exception('the file ',fileList[i],'is not a valid file')
  if i%1000==0: print(i,' images out of', len(fileList), ' are loaded')
assert len(images)==y.size
X=np.asarray(images)
print('Loading images succesful')
print('Data shape: ',X.shape)
print('Labels shape: ',y.shape)
#------------------------split data---------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=1)
print('X_train: ',X_train.shape)
print('y_train: ',y_train.shape)
print('X_test: ',X_test.shape)
print('y_test: ',y_test.shape)
print('X_val: ',X_val.shape)
print('y_val: ',y_val.shape)


#-------------------Define Model------------------------------------------------
model = keras.Sequential()

# Convolutional layer and maxpool layer 1
model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(460,700,3)))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 2
model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 3
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 4
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# This layer flattens the resulting image array to 1D array
model.add(keras.layers.Flatten())

# Hidden layer with 512 neurons and Rectified Linear Unit activation function 
model.add(keras.layers.Dense(512,activation='relu'))

# Output layer with single neuron which gives 0 for benign or 1 for malign
#Here we use sigmoid activation function which makes our model output to lie between 0 and 1
model.add(keras.layers.Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

batch_size = 32

model.summary()

#-----------------Define callback-----------------------------------------------
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/ImageClassificationBreastCancerCNN/checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

#model.load_weights(checkpoint_path)
#----------------Train the model------------------------------------------------
model.fit(X_train, 
          y_train,  
          epochs=10,
          validation_data=(X_val, y_val),
          callbacks=[cp_callback])  # Pass callback to training



    
    


0  images out of 622  are loaded
Loading images succesful
Data shape:  (622, 460, 700, 3)
Labels shape:  (622,)
X_train:  (372, 460, 700, 3)
y_train:  (372,)
X_test:  (125, 460, 700, 3)
y_test:  (125,)
X_val:  (125, 460, 700, 3)
y_val:  (125,)
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 458, 698, 32)      896       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 229, 349, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 227, 347, 64)      18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 113, 173, 64)     0         
 g2D)                                                            
        

In [ ]:
#evaluate model
loss, acc = model.evaluate(X_test, y_test, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

4/4 - 21s - loss: 166.1185 - accuracy: 0.3280 - 21s/epoch - 5s/step
Restored model, accuracy: 32.80%
